# Train a Network
本章节重点如何使用Pytorch 训练一个网络，主要会从随机梯度下降、损失函数、优化器、网络结构以及训练策略几方面展开

## 随机梯度下降
在网络训练过程中，我们会设定一个损失函数来约束我们的训练过程，从而使得网络参数朝着最小化损失函数的方向去训练。因此，我们需要用到梯度下降类的算法，来确定网络参数的优化方向。

梯度下降法，通过函数当前点的对应梯度的反方向，使用规定的步长(学习率)进行迭代搜索，从而找到函数的极小值点。

随机梯度下降法(SGD)，是在梯度下降法的基础上，对每个样本点的梯度进行平均，从而减少局部最小值的影响，提高训练效果。

SGD的缺点有以下：学习率难以确定，因为网络不同的训练阶段、不同结构的网络参数很难有一个统一的学习率，针对这种缺点，可以采用变化的学习率进行训练；另外，SGD容易陷入局部最小值点，从而无法跳出，这个时候可以使用动量法进行优化。

动量法，通过模拟物体运动时的惯性更新网络参数，即更新参数时会在一定程度上考虑之前参数的更新方向，同时利用当前batch得到的梯度，将二者结合进行更新.

Nesterov动量法，是在动量法的基础上，对参数更新公式进行了修改，因为之前累积的动量并不会直接影响当前batch的梯度计算，Nesterov动量法在更新参数时，会将累积的动量考虑在内，从而影响当前batch的梯度。

## 损失函数
深度学习中，网络的直接作用对象是损失函数。损失函数就是用来表示预测与实际数据之间的差距程度。一个最优化问题的目标就是将损失函数最小化。
Pytorch中的`nn`模块中提供了多种可以直接使用的深度学习损失函数，如交叉熵、均方误差等，常见的损失函数如下所示：
1. `nn.MSELoss`：均方误差； L = 1/n Σ(yi - ŷi)²,其中，L是损失函数的值，n是样本数量，yi是第i个样本的真实值，ŷi是第i个样本的预测值。Σ表示对所有样本进行求和。
2. `nn.L1Loss`：绝对误差； L = 1/n Σ|yi - ŷi|
3. `nn.CrossEntropyLoss`: 交叉熵损失：L = 1/n Σ|yi*log(ŷi)|,注意，`nn.CrossEntropyLoss`函数在内部同时计算 `log_softmax`(将输出转化为概率分布)和`nll_loss`,因此网络最后一层不需要添加`softmax`层，直接输出原始`logits`即可
4. `nn.BCELoss`: 二元交叉熵损失，常用于二分类问题。
5. `nn.NLLLoss`: 负对数似然损失，常用于多分类问题。
6. `nn.SmoothL1Loss`: 平滑L1损失

## 优化器

Pytorch中自带内置的优化器，如Adam，SGD、RMSprop等，可以直接调用，具体如下所示：
1. `torch.optim.SGD(model.parameters(), lr=0.1)` 随机梯度下降算法
2. `torch.optim.Adam(model.parameters(), lr=0.001)` Adam算法
3. `torch.optim.RMSprop(model.parameters(), lr=0.01)` RMSprop算法

其中 `torch.optim.Optimizer` 是优化器的基类，主要有以下几种使用方式
1. `zero_grad()`：清空梯度
2. `step()`：更新参数
3. `state_dict()`：获取当前优化器的状态字典
4. `load_state_dict(state_dict)`：从状态字典加载参数

In [ ]:
# 建立一个测试网络
import torch.nn as nn
import torch.optim as optim
class TestNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        
        # 定义隐藏层hidden1
        self.hidden1 = nn.Linear(784, 256)
        # 定义预测回归层
        self.predict = nn.Linear(256, 10)
    
    # 定义前向传播路径
    def forward(self, x):
        x = self.hidden1(x)
        out = self.predict(x)
        return out
# 实例化一个网络
net = TestNet()

# 定义优化器 
# 为每一层定义统一的学习率
optimizer = optim.Adam(net.parameters(), lr=0.01)

# 为每一层定义不同的学习率
optimizer = optim.Adam([
    {'params': net.hidden1.parameters(), 'lr': 0.01},
    {'params': net.predict.parameters(), 'lr': 0.1},
])

# 对目标函数进行优化的通常格式
for input,target in dataset:
    optimizer.zero_grad() # 清空上一步的残余更新参数值
    output = net(input) # 得到预测值
    loss = loss_func(output, target) # 计算损失函数
    loss.backward() # 误差反向传播，
    optimizer.step() # 参数更新

## 网络定义及训练

网络的定义以及训练主要包括数据预处理、优化器选择、损失函数、网络搭建等多个步骤，以波士顿房价预测为例，搭建整个网络结构

In [ ]:
# 导入需要的包
import torch
import torch.nn as nn
from torch.optim import SGD # 随机梯度下降优化器
import torch.utils.data as Data # 数据加载器
from sklearn.datasets import fetch_openml # 数据导入
from sklearn.preprocessing import StandardScaler # 数据标准化
import pandas as pd # 数据处理
import numpy as np
import matplotlib.pyplot as plt # 画图

In [ ]:
# 定义网络

# 使用继承Module的方式定义网络
class MLPmodel(nn.Module):
    def __init__(self):
        super(MLPmodel,self).__init__()
        
        # 定义隐藏层并用nn.Sequential()将网络层组合起来
        self.hidden = nn.Sequential(
            nn.Linear(13, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU()
        )
        self.predict = nn.Linear(10, 1)
        
    # 前向传播过程
    def forward(self, x):
        x = self.hidden(x)
        output = self.predict(x)
        return output

In [ ]:
# 读取Boston数据集,读取进来的可能是numpy数组或者DataFrame
boston = fetch_openml(name='boston', version=1, as_frame=True)
boston_X, boston_y = boston.data.to_numpy(), boston.target.to_numpy()

# 标准化处理
scaler = StandardScaler(with_mean=True, with_std=True)
boston_X = scaler.fit_transform(boston_X)

# 数据转换为tensor
train_X  = torch.from_numpy(boston_X.astype(np.float32))
train_y = torch.from_numpy(boston_y.astype(np.float32))

# 数据打包
train_dataset = Data.TensorDataset(train_X, train_y)

#定义数据加载器
train_loader = Data.DataLoader(
    dataset=train_dataset,      # 加载的数据集
    batch_size=64,              # 批量大小
    shuffle=True,               # 设置为True，数据将混洗
)

In [ ]:
# 实例化网络
net = MLPmodel()

# 定义优化器
optimizer = SGD(net.parameters(), lr=0.01)

# 定义损失函数
loss_func = nn.MSELoss()

# 记录训练过程中损失值的变化
loss_list = []

# 进行训练
for epoch in range(30):
    for step, (batch_x, batch_y) in enumerate(train_loader):
        # 计算输出
        output = net(batch_x)
        # 计算损失
        loss = loss_func(output, batch_y)
        # 清空上一步的残余更新参数值
        optimizer.zero_grad()
        # 误差反向传播，计算参数更新值
        loss.backward()
        # 参数更新
        optimizer.step()
        # 记录损失
        loss_list.append(loss.item())

In [ ]:
# 损失可视化
plt.figure()
plt.plot(loss_list,'r-')
plt.title('loss')
plt.show()

In [ ]:
# 保存整个模型
torch.save(net, 'net.pkl')

# 只保存模型参数
torch.save(net.state_dict(), 'net_params.pkl')

In [ ]:
# 使用保存的模型进行预测
# 加载模型
net = torch.load('net.pkl')

# 设置为评估模式，避免Dropout等层发生改变
net.eval()

# 处理输入
x = train_X[0]

# 预测输出
output = net(x)

# 打印输出的值
print(output.item())

总结：
对于网络的定义及训练，上述代码只是简单的搭建一个两层的MLP，但是真实情况下，模型会更为复杂，而且损失函数、学习率这些也需要多次迭代。

## 训练策略
网络训练时，可能会出现因为过于精确匹配训练数据集，从而失去泛化能力，导致模型不能很好的去拟合其他数据或者预测未来的数值。在深度学习中，将在训练数据集上获得很高的识别精度，但是在测试集上效果往往不理想的情况称之为过拟合，使用更好的训练策略可以有效防止过拟合
1. 增加数据量
2. 合理的数据切分。将数据集按照8:1:1或者其他比例切分为训练集、验证集、测试集，通过训练集训练网络参数，验证集用来监督模型是否过拟合，测试集用来测试模型的泛化能力
3. 正则化。通过添加范数惩罚对需要训练的参数进行约束，l1范数目的是将参数的绝对值最小化，l2范数将参数的平方和最小化。使用l1范数，网络趋向于使用更少的参数，而其他参数都是0，从而增加网络稀疏性；使用l2范数，则会趋向于使用更多的参数，但是这些参数接近于0
4. Dropout。在每个训练批次中，通过忽略一定百分比二点神经元数量，减轻网络的过拟合现象
5. 早停法。在训练过程中，通过验证集上的误差来选择最好的模型，当验证集上的误差不再下降时，停止训练

## 学习率调整
由于在网络训练中，学习率不是固定不变的，因此Pytorch在 torch.optim.lr_scheduler 模块中提供了内置的优化器学习率调整方式,常用几种如下：
1. `lr_scheduler.LambdaLR(optimizer, lr_lambda,last_epoch=-1)`,其中`last_epoch` 用于设置合适开始调整学习率的轮数，`lr_lambda` 是一个函数，接收当前迭代轮数作为输入，返回一个调整系数。
2. `lr_scheduler.StepLR(optimizer, step_size, gamma=0.1, last_epoch=-1)`, 步进式学习率调整，每`step_size`个epoch，将学习率乘以`gamma`。
3. `lr_scheduler.MultiStepLR(optimizer, milestones, gamma=0.1, last_epoch=-1)`, 多步学习率调整，在`milestones`指定的epoch进行调整，每次调整将学习率乘以`gamma`。
4. `lr_scheduler.ExponentialLR(optimizer, gamma, last_epoch=-1)`,指数衰减式学习率调整，每次调整将学习率乘以`gamma`。
5. `lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=False, threshold=1e-4, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)`，当验证集指标不再下降时，将学习率乘以`factor`。
6. `lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1)`，余弦退火学习率调整，在`T_max`个epoch内，将学习率从初始值线性调整为`eta_min`。